Yonghwan Kim A11746276

In [1]:
# pyspark install
!pip install pyspark

     |████████████████████████████████| 212.4 MB 76 kB/s 
     |████████████████████████████████| 198 kB 68.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=3e2016d8c3f786368794b25b25da244d0724ae40880be958b390c15b2f42d1d2
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
import sys
from itertools import chain, combinations # iteration tool
 
from pyspark import SparkContext, SparkConf

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark - CS5683 - Project-1")

In [78]:
views = '/content/drive/MyDrive/views.txt'
get_recommendations = '/content/drive/MyDrive/get_recommendations.txt'
mappings = '/content/drive/MyDrive/mappings.txt'

# support threshold
s = 3
# Iteration
k = 2
# confidence
c = 0.6

# remove unecessary data
def associationRule(x):
    x[0].remove(x[1])
    return x
# find all the subsets of basket with given range
def powerset(iterable, n):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(n, n+1))
# changing to tuple to digest iterable type
def distribute(iterable):
    return tuple(iterable)
# make candidate combinations with sigleton broadcast and k-th combinations
def comb(x):
    combined = []
    for single in singleBroadcast.value:        
          if not set(single).issubset(set(x)):
             combined.append(set(single).union(set(x)))
    return combined

# launch view.txt to spark
fileRDD = sc.textFile(views)
wordsRDD = fileRDD.flatMap(lambda line: line.split("\n"))
# create singleton with it's support value
itemsRDD = wordsRDD.flatMap(lambda line: line.split("::"))
singletonRDD = itemsRDD.map(lambda word: (word,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] >= s)

# broadcast singleton
singleBroadcast = sc.broadcast(singletonRDD.keys().map(lambda i:tuple((i,))).collect())
# create basket rdd
basketRDD = wordsRDD.map(lambda line: line.split("::")).map(lambda word: frozenset(word))
# initialize frequent itemsets rdd empty
frequentItemsetRDD = sc.emptyRDD()
# intialize candidate itemsets rdd with singleton keys
candidateItemsetRDD = singletonRDD.keys()

# for loop to create candidate itemsets and its support value to filter out later.
while k <= 4:      
      # create candidates using broadcasted singleton and k-th list.
      if k == 2:
          candidateRDD = candidateItemsetRDD.flatMap(lambda a : comb((a,))).map(lambda item: (frozenset(item) , 1)).reduceByKey(lambda a,b: a+b)
      else:
          candidateRDD = candidateItemsetRDD.flatMap(lambda a : comb(a)).map(lambda item: (frozenset(item) , 1)).reduceByKey(lambda a,b: a+b)
      # find support value of k-th list from basket
      scountbasketRDD = basketRDD.flatMap(lambda word: distribute(powerset(word, k))).map(lambda w: (frozenset(w), 1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] >= s)
      # find matching candidates
      frequentCandidateRDD = candidateRDD.join(scountbasketRDD).map(lambda item: (item[0], item[1][1]))
      # combine current k itemsets with previous list
      frequentItemsetRDD = frequentItemsetRDD.union(frequentCandidateRDD)
      # prepare k itemsets for next iteration step
      candidateItemsetRDD = frequentCandidateRDD.keys()
      # increase k to go next iteration
      k = k + 1

In [ ]:
# generate association rules
associationRDD = frequentItemsetRDD.cartesian(singletonRDD).filter(lambda i : any(i[1][0] == x for x in i[0][0]))
# calculate confidence level and filter the rules that are lesser than confidence threshold 60%
confiRDD = associationRDD.map(lambda i : (i, (i[0][1]/i[1][1]))).filter(lambda i : i[1] >= c)
# sort all association 
confiRDD = confiRDD.sortBy(lambda x: x[1], False)

finalAssociationRDD = confiRDD.map(lambda i : (list(i[0][0][0]), i[0][1][0], i[1]*100)).map(associationRule)
# save the association rules in one output file
finalAssociationRDD.coalesce(1,True).saveAsTextFile("/content/association")

In [76]:
#Bonus Point
recomRDD = sc.textFile(get_recommendations).flatMap(lambda line: line.split("\n")).map(lambda line: line.split("::"))
recommBroadcast = sc.broadcast(recomRDD.collect())
associBroadcast = sc.broadcast(finalAssociationRDD.collect())

# find 3 movies for each watch list
temp = []

for rec in recommBroadcast.value:
    # print(set(rec))
    recommd = 0
    rectemp = []
    # broadcasted association rules
    for a in associBroadcast.value:
          if recommd == 3:
            break
          if set(a[0]).issubset(set(rec)):
              # recommand 3 movies that are not already recommanded and not already watched
              if not set([a[1]]).issubset(set(rec)) and not set([a[1]]).issubset(set(rectemp)): 
                  rectemp.append(a[1])
                  recommd = recommd + 1
    temp.append(rectemp)
# print(temp)
# find actual name of the moives
mappingRDD = sc.textFile(mappings).flatMap(lambda line: line.split("\n")).map(lambda line: line.split("::"))
mappingBroadcast = sc.broadcast(mappingRDD.collect())

title = []
for t in temp:
    indTitle = []
    for l in t:
        for m in mappingBroadcast.value:
            if l == m[0]:
                indTitle.append(m[1])
    title.append(indTitle)
# final result
print(title)

[[], [], [], ['Rocky (1976)', 'Play it to the Bone (1999)', 'Grand Day Out, A (1992)'], ['Exorcist II: The Heretic (1977)', 'Secret of NIMH, The (1982)', 'eXistenZ (1999)'], ['Hate (Haine, La) (1995)', 'Cutthroat Island (1995)'], ['Nixon (1995)', 'Suicide Kings (1997)', 'Rocky (1976)'], ['Better Living Through Circuitry (1999)', 'Gnome-Mobile, The (1967)', "Nightmare on Elm Street Part 2: Freddy's Revenge, A (1985)"], [], ['Tough Guys (1986)', 'Rough Magic (1995)', 'Zeus and Roxanne (1997)']]
